# Ground State Energy Estimation for Photosynthesis with Quantum Circuits

Artificial photosynthesis mimics natural photosynthesis by
absorbing solar light and splitting water into O2, protons (H+) and electrons (e).

The electrons extracted from water can reduce protons or CO2
to produce energy carrier fuels such as H2 or hydrocarbons

Water oxidation reaction
2H2O -> O2 + 4(H+) + 4e

Proton reduciton reaction:
2(H+) + 2e -> H2

CO2 reduciton reactions:
- CO2 + 2(H+) + 2e -> CO + H2O    (CO2 Reduction 1)

- CO2 + 6(H+) + 6e -> CH3OH + H2O (CO2 Reduction 2)

- CO2 + 8(H+) + 8e -> CH4 + 2H2O  (CO2 Reduction 3)

Typical examples of water oxidation:
Co_4O_4 catalysis (https://pubs.acs.org/doi/10.1021/ja202320q)

In [1]:
import sys
import re
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from openfermion.linalg import get_ground_state, get_sparse_operator

import networkx as nx
import openfermion as of
import numpy as np
import matplotlib.pyplot as plt
import time
import cirq
import os
from pyLIQTR.utils.utils import count_T_gates, open_fermion_to_qasm
from pyLIQTR.gate_decomp.cirq_transforms import clifford_plus_t_direct_transform
from cirq.contrib.qasm_import import circuit_from_qasm
from pyLIQTR.GSE.GSE import GSE
from pyLIQTR.QSP.qsp_helpers import qsp_decompose_once

In [2]:
def extract_number(string):
    number = re.findall(r'\d+', string)
    return int(number[0]) if number else None

def count_gates(cpt_circuit):
    count = 0
    for moment in cpt_circuit:
        for operator in moment:
            count += 1
    return count

def num_fermion_qubits(ham):
    n_qubits = 0
    for term in ham.terms:
        for op in term:
            qubit = op[0]
            if qubit >= n_qubits:
                n_qubits = qubit+1
    return n_qubits

def estimate_gse(circuit, outdir, circuit_name="gse_circuit"):
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    
    subcircuit_counts = dict()
    t_counts = dict()
    clifford_counts = dict()
    gate_counts = dict()
    subcircuit_depths = dict()
    
    outfile_data = outdir+circuit_name+"_high_level.dat"
    
    for moment in circuit:
        for operation in moment:
            gate_type = type(operation.gate)
            if gate_type in subcircuit_counts:
                subcircuit_counts[gate_type] += 1
            else:
                decomposed_circuit = qsp_decompose_once(qsp_decompose_once(cirq.Circuit(operation)))
                cpt_circuit = clifford_plus_t_direct_transform(decomposed_circuit)
                
                subcircuit_counts[gate_type] = 1
                subcircuit_depths[gate_type] = len(cpt_circuit)
                t_counts[gate_type] = count_T_gates(cpt_circuit)
                gate_counts[gate_type] = count_gates(cpt_circuit)
                clifford_counts[gate_type] = gate_counts[gate_type] - t_counts[gate_type]
                
                
    total_gate_count = 0
    total_gate_depth = 0
    total_T_count = 0
    total_clifford_count = 0
    for gate in subcircuit_counts:
        total_gate_count += subcircuit_counts[gate] * gate_counts[gate]
        total_gate_depth += subcircuit_counts[gate] * subcircuit_depths[gate]
        total_T_count += subcircuit_counts[gate] * t_counts[gate]
        total_clifford_count += subcircuit_counts[gate] * clifford_counts[gate]
    with open(outfile_data, 'w') as f:
        total_gate_count 
        f.write(str("Logical Qubit Count:"+str(len(circuit.all_qubits()))+"\n"))
        f.write(str("Total Gate Count:"+str(total_gate_count)+"\n"))
        f.write(str("Total Gate Depth:"+str(total_gate_depth)+"\n"))
        f.write(str("Total T Count:"+str(total_T_count)+"\n"))
        f.write(str("Total Clifford Count:"+str(total_clifford_count)+"\n"))
        f.write("Subcircuit Info:\n")
        for gate in subcircuit_counts:
            f.write(str(str(gate)+"\n"))
            f.write(str("Subcircuit Occurrences:"+str(subcircuit_counts[gate])+"\n"))
            f.write(str("Gate Count:"+str(gate_counts[gate])+"\n"))
            f.write(str("Gate Depth:"+str(subcircuit_depths[gate])+"\n"))
            f.write(str("T Count:"+str(t_counts[gate])+"\n"))
            f.write(str("Clifford Count:"+str(clifford_counts[gate])+"\n"))$

In [17]:
#Water oxidation via Co4O4 catalyst.
with open("data/water_oxidation_Co4O4.txt") as f:
    r""" Proposed pathway of Co dimer catalyst
    J. Phys. Chem. C 2015, 119, 11072−11085
    """
    pathway0 = [5, 10, 28, 29, 30, 31, 32, 33]
    pathway1 = [2, 1, 14, 15, 16, 17, 18, 19]
    pathway2 = [3, 1, 14, 15, 16, 20, 21, 22, 23]
    pathway3 = [27, 1, 14, 15, 16, 24, 25, 26]
    data = f.readlines()

    coordinates_pathway = []
    for k, line in enumerate(data):
        if "Multiplicity" in line:
            geo_name = line.split()[0]
            # Use a regular expression to extract the multiplicity value
            match = re.search(r'Multiplicity=(\d+)', line)
            if match:
                multiplicity = int(match.group(1))  # Convert the string to an integer


            coords = data[k+1]
            coords = coords.split()
            nat = len(coords) // 5
            #print(f"test, {len(coords)} {nat}")
            coords_list = []
            coords_list.append([nat, 0, multiplicity])

            for i in range(0, len(coords), 5):
                aty = coords[i]
                xyz = [float(coords[i+j]) for j in range(2,5)]
                coords_list.append([aty, xyz])

                #print(f"{aty:3} {xyz[0]:10.6f} {xyz[1]:10.6f} {xyz[2]:10.6f}")
                #print(f"{aty:3}", end=" ")
                #for x in xyz:
                #    print(f"{x:10.6f}", end=" ")
                #print()

            print(f"{geo_name} Multiplicity: {multiplicity}  total no. of atoms={nat}")
            order = extract_number(geo_name)
            #print(coords_list)
            if order is not None and order in pathway0:
                #print(f"{geo_name} is in pathway 1")
                coordinates_pathway.append(coords_list)


# Set calculation parameters.
run_scf = 1
run_mp2 = 0
run_cisd = 0
run_ccsd = 0
run_fci = 0

# Set molecule parameters.
basis = 'sto-3g'
multiplicity = 1
n_points = 40
bond_length_interval = 3.0 / n_points

print("\n Generating the electronic Hamiltonain along the reaction pathway!\n")
active_space_frac = 5 # 1 over n

molecular_hamiltonians = {}
count = 0

if len(coordinates_pathway) > 0:
    # generate the Hamiltonians
    for coords in coordinates_pathway:
        nat, charge, multi = [int(coords[0][j]) for j in range(3)]
        print(f"\nGenerating the qubit Hamiltonian for the molecule: \n {coords}\n")
        #print(f"\nNatoms={nat} charge={charge} multiplicity={multi}")

        # set molecular geometry in pyscf format
        basis = "STO3G"
        geometry = []
        for k, coord in enumerate(coords[1:]):
            coord_str = ' '.join(map(str, coord[1]))
            if k == nat-1:
                coord_str = f"{coord[0]} {coord_str}"
            else:
                coord_str = f"{coord[0]} {coord_str};\n"
            atom = (coord[0], tuple(coord[1]))
            geometry.append(atom)

        molecule = MolecularData(geometry, basis, multi, description="catalyst")

        # Run pyscf.
        molecule = run_pyscf(molecule,
                             run_scf=run_scf,
                             run_mp2=run_mp2,
                             run_cisd=run_cisd,
                             run_ccsd=run_ccsd,
                             run_fci=run_fci)

        print(f"number of orbitals          = {molecule.n_orbitals}")
        print(f"number of electrons         = {molecule.n_electrons}")
        print(f"number of qubits            = {molecule.n_qubits}")
        nocc = molecule.n_electrons // 2
        nvir = molecule.n_orbitals - nocc
        sys.stdout.flush()

        # get molecular Hamiltonian
        active_space_start =  nocc - nocc // active_space_frac # start index of active space
        active_space_stop = nocc + nvir // active_space_frac   # end index of active space

        print(f"active_space start = {active_space_start}")
        print(f"active_space stop  = {active_space_stop}")
        sys.stdout.flush()

        molecular_hamiltonian = molecule.get_molecular_hamiltonian(
            occupied_indices=range(active_space_start),
            active_indices=range(active_space_start, active_space_stop)
        )
        print(type(molecular_hamiltonian))
        molecular_hamiltonians[count] = molecular_hamiltonian
        count += 1

D1 Multiplicity: 1  total no. of atoms=26
D2 Multiplicity: 1  total no. of atoms=26
D3 Multiplicity: 1  total no. of atoms=26
D4 Multiplicity: 1  total no. of atoms=26
D5 Multiplicity: 1  total no. of atoms=26
D6 Multiplicity: 1  total no. of atoms=26
D7 Multiplicity: 1  total no. of atoms=26
D8 Multiplicity: 1  total no. of atoms=26
D9 Multiplicity: 1  total no. of atoms=26
D10 Multiplicity: 1  total no. of atoms=26
D11 Multiplicity: 1  total no. of atoms=26
D12 Multiplicity: 1  total no. of atoms=26
D13 Multiplicity: 1  total no. of atoms=26
D14 Multiplicity: 2  total no. of atoms=25
D15 Multiplicity: 3  total no. of atoms=24
D16 Multiplicity: 2  total no. of atoms=23
D16' Multiplicity: 4  total no. of atoms=23
D17 Multiplicity: 2  total no. of atoms=26
D18 Multiplicity: 3  total no. of atoms=25
D19 Multiplicity: 1  total no. of atoms=23
D20 Multiplicity: 2  total no. of atoms=23
D21 Multiplicity: 2  total no. of atoms=26
D22 Multiplicity: 2  total no. of atoms=29
D23 Multiplicity: 3

KeyboardInterrupt: 

In [ ]:
for i in molecular_hamiltonians:
    molecular_hamiltonian = molecular_hamiltonians[i]
    trotter_order = 2
    trotter_steps = 1
    n_qubits = molecular_hamiltonian.n_qubits
    #note that we would actually like within chemical precision
    #which should take > 10 bits of precision, it just takes a 
    #really long time to run so a scaling argument will be needed
    bits_precision = 1
    
    gse_args = {
        'trotterize' : True,
        'mol_ham'    : molecular_hamiltonian,
        'ev_time'    : 1,
        'trot_ord'   : trotter_order,
        'trot_num'   : trotter_steps
    }

    E_min = -4000
    E_max = -3000
    
    init_state = [0] * n_qubits
    
    print("starting")
    t0 = time.perf_counter()
    gse_inst = GSE(
        precision_order=bits_precision,
        init_state=init_state,
        E_max = E_max,
        E_min = E_min,
        include_classical_bits=False, # Do this so print to openqasm works
        kwargs=gse_args)
    t1 = time.perf_counter()
    print("Co4O4 time to generate high level number " +str(i)+ ": ", t1 - t0)
    gse_circuit = gse_inst.pe_inst.pe_circuit
    
    print("Estimating Co4O4 circuit " + str(i))
    t0 = time.perf_counter()
    estimate_gse(gse_circuit, outdir="GSE/", circuit_name="Co4O4_"+str(i))
    t1 = time.perf_counter()
    print("Time to estimate Co4O4:", t1-t0)

starting
Co4O4 time to generate high level number 0:  27.952823110999816
Estimating Co4O4 circuit 0


In [16]:
test_ham = molecular_hamiltonians[0]

In [18]:
len(test_ham)

TypeError: object of type 'InteractionOperator' has no len()